**Import Libraries Needed **

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.metrics import top_k_categorical_accuracy
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.utils import load_img, img_to_array
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay




In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
labels = []
image_path = []
for dirname, _, filenames in os.walk('/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/train'):
    for filename in filenames:
        if filename.endswith('.jpg'):
            image_path.append(os.path.join(dirname, filename))
            labels.append(os.path.basename(os.path.normpath(dirname)))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
model = ResNet50(include_top=False, input_shape=(224,224,3), weights = "/kaggle/input/resnet-weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

## defining early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

#freezing layers
for layer in model.layers:
    layer.trainable = False

#add new classifier layers 

flat1 = Flatten()(model.output)
class1 = Dense(1024, activation = 'relu')(flat1)
drop_layer = Dropout(0.2)(class1)
output = Dense(196, activation = 'softmax')(drop_layer)
model = Model(inputs = model.inputs, outputs = output)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='sgd', 
              loss='categorical_crossentropy', 
              metrics = ['accuracy', 'top_k_categorical_accuracy'])

In [ ]:
## Attempting Data Augmentation
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True, width_shift_range=0.1, height_shift_range=0.1)

## no data augmentation
#data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

#preprocessing
train_folder="../input/stanford-car-dataset-by-classes-folder/car_data/car_data/train"
test_folder="../input/stanford-car-dataset-by-classes-folder/car_data/car_data/test"

#grabbing different images
train_generator = data_generator.flow_from_directory(train_folder, target_size=(224, 224), class_mode='categorical')
test_generator = data_generator.flow_from_directory(test_folder, target_size=(224, 224), class_mode='categorical')

history=model.fit(train_generator, steps_per_epoch=50, validation_data=test_generator, validation_steps=1, epochs=100, callbacks=[early_stop])


In [ ]:
history.history.keys()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

# Calculate the trend line
accuracy_trend = np.poly1d(np.polyfit(range(len(history.history['accuracy'])), history.history['accuracy'], 1))
val_accuracy_trend = np.poly1d(np.polyfit(range(len(history.history['val_accuracy'])), history.history['val_accuracy'], 1))

# Plot the trend lines
plt.plot(range(len(history.history['accuracy'])), accuracy_trend(range(len(history.history['accuracy']))), '--')
plt.plot(range(len(history.history['val_accuracy'])), val_accuracy_trend(range(len(history.history['val_accuracy']))), '--')

plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch(s)')
plt.legend(['Train', 'Test', 'Train Trend', 'Test Trend'], loc='upper left')
plt.show()

In [ ]:
# create a plot for the accuracies
history.history.keys()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['val_top_k_categorical_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch(s)')
plt.legend(['Train', 'Test', 'Top 5'], loc='upper left')
plt.show()

# create a plot for the loss functions
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch(s)')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:

car_classes = list(test_generator.class_indices.keys())

## Select Car
am_general_hummer_suv_2000 = "/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test/AM General Hummer SUV 2000/00076.jpg"

# function that takes the car and turns it into useable format for predictions
def processing(car_type):
    img = load_img(car_type, 
                     target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    car = np.vstack([x])
    return(car)

# function that will decode the predictions of given input and return top 5
def decoded_preds(car):
    prediction = model.predict(car)
    zip_list = list(zip(car_classes,prediction[0]))
    result = []

    for (car, pred) in zip_list:
        result.append((car,pred))   
    top_five = sorted(result, key=lambda a: a[1], reverse=True)[:5]
    return(top_five)
print("results for am_general_hummer_suv_2000" )
decoded_preds(processing(am_general_hummer_suv_2000))

In [ ]:
bentley_arnage_sedan_2009 = "/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test/Bentley Arnage Sedan 2009/00277.jpg"
print("results for bentley_arnage_sedan_2009")
decoded_preds(processing(bentley_arnage_sedan_2009))


In [ ]:
dodge_charger_sedan_2012 = "/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test/Dodge Charger Sedan 2012/00094.jpg"
print("results for dodge_charger_sedan_2012")
decoded_preds(processing(dodge_charger_sedan_2012))
    